# Portfolio's Building #

### Rebalancing Weights ###

In [241]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Optiminization
from scipy.optimize import minimize

# Pretty Notation
from IPython.display import display, Math

In [242]:
def import_financial_data(
    ticker: str
):

    # Check the ticker for Upper Cases
    ticker = ticker if ticker.isupper() else ticker.upper()

    # Import data
    df = pd.read_csv(rf"..\stocks\{ticker}.csv")

    # Set the Index
    df = df.set_index('Date')
    df.index = pd.to_datetime(df.index)

    df_useful_data = df[['Open Price', 'High Price', 'Low Price', 'Close Price', 'Adjusted_close']]

    df_useful_data = df_useful_data.rename(columns={
        "Open Price":"open",
        "High Price":"high",
        "Low Price":"low",
        "Close Price":"close",
        "Adjusted_close":"adjusted_close",
    })

    # Drop NaN's
    df_useful_data.dropna(inplace = True)

    return df_useful_data.loc["2013-01-01":]

In [243]:
# Import Data

# Apple Data
df_1 = import_financial_data("AAPL")

# Amazon Data
df_2 =  import_financial_data("AMZN")

# Meta Data
df_3 =  import_financial_data("META")

# Microsoft Data
df_4 =  import_financial_data("MSFT")

# Nvidia Data
df_5 =  import_financial_data("NVDA")

In [244]:
# Create the joint dataframe

df_data = pd.DataFrame()

df_data['AAPL'] = df_1['close']
df_data['AMZN'] = df_2['close']
df_data['META'] = df_3['close']
df_data['MSFT'] = df_4['close']
df_data['NVDA'] = df_5['close']

df_data = df_data.dropna()

df_returns = df_data.pct_change(1).mul(100)
df_returns = df_returns.apply(lambda x: x.fillna(x.mean()), axis=0)

df_returns

In [245]:
df_returns.mean()

In [246]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_returns['AAPL'].cumsum(), label='APPL Returns', color='blue', alpha=0.7)
plt.plot(df_returns['AMZN'].cumsum(), label='AMZN Returns', color='green', alpha=0.7)
plt.plot(df_returns['META'].cumsum(), label='META Returns', color='red', alpha=0.7)
plt.plot(df_returns['MSFT'].cumsum(), label='MSFT Returns', color='orange', alpha=0.7)
plt.plot(df_returns['NVDA'].cumsum(), label='NVDA Returns', color='purple', alpha=0.7)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [247]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_returns['AAPL'].ewm(span=252, adjust=False).mean(), label='APPL Returns', color='blue', alpha=0.7)
plt.plot(df_returns['AMZN'].ewm(span=252, adjust=False).mean(), label='AMZN Returns', color='green', alpha=0.7)
plt.plot(df_returns['META'].ewm(span=252, adjust=False).mean(), label='META Returns', color='red', alpha=0.7)
plt.plot(df_returns['MSFT'].ewm(span=252, adjust=False).mean(), label='MSFT Returns', color='orange', alpha=0.7)
plt.plot(df_returns['NVDA'].ewm(span=252, adjust=False).mean(), label='NVDA Returns', color='purple', alpha=0.7)

# Config
plt.title('Annual Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [248]:
# Create the DataFrame with average returns

df_mean_returns = df_returns.apply(lambda x: x.ewm(span=252, adjust=False).mean().mul(252), axis=0)

df_mean_returns

In [249]:
# Let us Calculate the Weights

def rolling_weights(
    returns, 
    desired_returns, 
    window=252, 
    rebalance_freq=126
):

    # Lists to Store Things
    weights_list = []
    dates = []

    for i in range(window, len(returns), rebalance_freq):
        past_returns = returns.iloc[i-window:i]  # Rolling Window
        
        # Mean and Covariance
        mu = past_returns.mean()
        Sigma = past_returns.cov()
        Sigma_inv = np.linalg.inv(Sigma)
        iota = np.ones(len(mu))

        # Markowitz Components
        A = mu @ Sigma_inv @ mu
        B = mu @ Sigma_inv @ iota
        C = iota @ Sigma_inv @ iota
        D = (A * C) - (B ** 2)

        w = (((desired_returns * C) - B) / D) * (Sigma_inv @ mu) + \
            ((A - (desired_returns * B)) / D) * (Sigma_inv @ iota)

        # Save weights and dates
        weights_list.append(w)
        dates.append(returns.index[i])

    # Create the DataFrame
    weights_df = pd.DataFrame(weights_list, index=dates, columns=returns.columns)

    # Expand the DataFrame
    weights_df = weights_df.reindex(returns.index, method='ffill')

    return weights_df.dropna()

In [250]:
# Create the DataFrames of Returns

df_weights = rolling_weights(df_returns, 0.2)

df_weights

In [251]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_weights['AAPL'], label='APPL Weights', color='blue', alpha=1)
plt.plot(df_weights['AMZN'], label='AMZN Weights', color='green', alpha=1)
plt.plot(df_weights['META'], label='META Weights', color='red', alpha=1)
plt.plot(df_weights['MSFT'], label='MSFT Weights', color='orange', alpha=1)
plt.plot(df_weights['NVDA'], label='NVDA Weights', color='purple', alpha=1)

# Config
plt.title('Weights Time Series')
plt.xlabel('Time Index')
plt.ylabel('Weights')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [252]:
# Common Index

common_index = df_returns.index.intersection(df_weights.index)  # Fechas en común
df_returns_reindex = df_returns.reindex(common_index)
df_weights = df_weights.reindex(common_index)

df_returns_reindex

In [253]:
# Create the Portfolio Returns

df_weighted_returns = df_returns_reindex * df_weights

df_weighted_returns

In [254]:
# Create the Portfolio Returns
df_returns_portfolio = df_returns.copy()

# Add the columns
df_returns_portfolio['Portfolio'] =  df_weighted_returns.sum(axis = 1)

df_returns_portfolio['Portfolio'].dropna()

In [255]:
# Time Series Graphs

df_plot = df_returns_portfolio.dropna()

plt.figure(figsize=(10, 6))
plt.plot(df_plot['AAPL'].cumsum(), label='APPL Returns', color='blue', alpha=0.7)
plt.plot(df_plot['AMZN'].cumsum(), label='AMZN Returns', color='green', alpha=0.7)
plt.plot(df_plot['META'].cumsum(), label='META Returns', color='red', alpha=0.7)
plt.plot(df_plot['MSFT'].cumsum(), label='MSFT Returns', color='orange', alpha=0.7)
plt.plot(df_plot['NVDA'].cumsum(), label='NVDA Returns', color='purple', alpha=0.7)
plt.plot(df_plot['Portfolio'].cumsum(), label='Portfolio Returns', color='black', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

### Define different rebalancing frequencies ###

In [257]:
# Create weights for different rebalancing frecuencies

df_weights_5d = rolling_weights(df_returns, 0.25, rebalance_freq=5)
df_weights_21d = rolling_weights(df_returns, 0.25, rebalance_freq=21)
df_weights_63d = rolling_weights(df_returns, 0.25, rebalance_freq=63)
df_weights_126d = rolling_weights(df_returns, 0.25, rebalance_freq=126)
df_weights_252d = rolling_weights(df_returns, 0.25, rebalance_freq=252)

In [258]:
# Create the Returns

df_weighted_returns_5d = df_returns_reindex * df_weights_5d
df_weighted_returns_21d = df_returns_reindex * df_weights_21d
df_weighted_returns_63d = df_returns_reindex * df_weights_63d
df_weighted_returns_126d = df_returns_reindex * df_weights_126d
df_weighted_returns_252d = df_returns_reindex * df_weights_252d

In [259]:
# Add the columns

df_returns_reindex['5d_port'] = df_weighted_returns_5d.sum(axis = 1)
df_returns_reindex['21d_port'] = df_weighted_returns_21d.sum(axis = 1)
df_returns_reindex['63d_port'] = df_weighted_returns_63d.sum(axis = 1)
df_returns_reindex['126d_port'] = df_weighted_returns_126d.sum(axis = 1)
df_returns_reindex['252d_port'] = df_weighted_returns_252d.sum(axis = 1)

In [260]:
# Time Series Graphs

plt.figure(figsize=(10, 6))
plt.plot(df_returns_reindex.cumsum(), label=df_returns_reindex.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

### Comparing Portfolios ###

In [262]:
def calculate_analytics(df_returns, risk_free_rate=0.0):
    # Trading Days in one Year
    ann_factor = 252  
    
    # Annualized Returns
    annualized_return = df_returns.mean() * ann_factor
    
    # Annualized Volatility
    annualized_std = df_returns.std() * np.sqrt(ann_factor)
    
    # Sharpe Ratio
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_std
    
    # Max Drawdown
    cumulative_returns = (1 + df_returns.div(100)).cumprod()
    rolling_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns / rolling_max) - 1
    max_drawdown = drawdown.min()

    # VaR at 95%
    var_95 = df_returns.quantile(0.05)

    # Create DF
    summary_df = pd.DataFrame({
        "Annualized Returns": annualized_return,
        "Annualized Volatility": annualized_std,
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown": max_drawdown,
        "VaR 95%": var_95
    })
    
    return summary_df

In [263]:
# Now the table
analytics_table = calculate_analytics(df_returns_reindex)

analytics_table